# Цель исследования

Компания стартап решила купить крупный сервис для чтения книг по подписке. Наша задача проанализировать данные которые мы имеем. База данных содержит информацию о книгах, оценках, рейтингах, издательствах и пользовательских обзорах книг.

Задания

   - Посчитайте, сколько книг вышло после 1 января 2000 года;
   - Для каждой книги посчитайте количество обзоров и среднюю оценку;
   - Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
   - Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
   - Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
   
Описание данных
   
    Таблица books
    Содержит данные о книгах:
      - book_id — идентификатор книги;
      - author_id — идентификатор автора;
      - title — название книги;
      - num_pages — количество страниц;
      - publication_date — дата публикации книги;
      - publisher_id — идентификатор издателя.
    
    Таблица authors
    Содержит данные об авторах:
      - author_id — идентификатор автора;
      - author — имя автора.

    Таблица publishers
    Содержит данные об издательствах:
      - publisher_id — идентификатор издательства;
      - publisher — название издательства;

    Таблица ratings
    Содержит данные о пользовательских оценках книг:
      - rating_id — идентификатор оценки;
      - book_id — идентификатор книги;
      - username — имя пользователя, оставившего оценку;
      - rating — оценка книги.
    
    Таблица reviews
    Содержит данные о пользовательских обзорах:
      - review_id — идентификатор обзора;
      - book_id — идентификатор книги;
      - username — имя автора обзора;
      - text — текст обзора.

In [27]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [28]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [29]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [30]:
#books
books = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(books), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [31]:
#authors
authors = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(authors), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [32]:
#ratings
ratings = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(ratings), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [33]:
#reviews
reviews = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(reviews), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [34]:
#publishers
publishers = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(publishers), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [35]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [68]:
count_books_avg_reviews_rating = '''
SELECT title AS name_books,
       COUNT(DISTINCT review_id) AS count_reviews,
       ROUND(AVG(ra.rating), 2) AS avg_rating
FROM books AS b
LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
LEFT JOIN reviews AS re ON ra.book_id=re.book_id
GROUP BY title, b.book_id
ORDER BY avg_rating DESC, count_reviews DESC
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(count_books_avg_reviews_rating), con = con)

,name_books,count_reviews,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


Выше мы вывели онформацию:
   - name_books - название книги
   - count_reviews - количество обзоров
   - avg_rating - средняя оценка

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [37]:
query = '''
WITH t1 AS (SELECT publisher,
                   COUNT(book_id) AS count_books
            FROM books AS b 
            JOIN publishers AS p ON b.publisher_id=p.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher)
SELECT *
FROM t1
WHERE count_books = (SELECT MAX(count_books) FROM t1)
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_books
0,Penguin Books,42


Наибольшее количество книг выпустило издательство Penguin Books и составило 42 книги.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [69]:
top_authors = '''
WITH t1 AS (SELECT b.book_id,
                   a.author,
                   COUNT(r.rating) AS count_rating,
                   AVG(r.rating) AS avg_rating
            FROM authors AS a
            JOIN books AS b ON a.author_id=b.author_id
            JOIN ratings AS r ON b.book_id=r.book_id
            GROUP BY b.book_id, a.author
            HAVING COUNT(r.rating) >= 50)
SELECT author,
       AVG(avg_rating) AS avg_rating_avg
FROM t1
GROUP BY author
ORDER BY avg_rating_avg DESC
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(top_authors), con = con)

,author,avg_rating_avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Автором с самым высоким средним рейтингом 4.28 стал J.K. Rowling/Mary GrandPré	

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [67]:
query = '''
SELECT ROUND(AVG(count_review), 2) AS avg_count_review
FROM (SELECT re.username,
             COUNT(DISTINCT re.review_id) AS count_review
      FROM reviews AS re
      WHERE re.username IN (SELECT ra.username
                            FROM ratings AS ra
                            GROUP BY ra.username
                            HAVING COUNT(ra.rating) > 48)
      GROUP BY re.username) AS t1;
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_count_review
0,24.0


Среднее количество обзоров от пользователей составило 24

Вывод:
   - В данных насчитывается 1000 книг;
   - В таблице 'count_books_avg_reviews_rating' для каждоый книги можно посмотреть количество обзоров и среднюю оценку;
   - Издательство Penguin Books выпустило самое большое количество книг - 42 книги;
   - В таблице 'top_authors' можно найти авторов с самым высоким средним рейтингом. Первую строчку занимает автор J.K. Rowling/Mary GrandPré с рейтингом 4.28;
   - В среднем количество обзоров от пользователей составляет 24.